In [42]:
import numpy as np
from tqdm import tqdm
from matplotlib import pyplot as plt
from keras.layers import Input, Reshape, Conv2D, Dense, Dropout, Flatten,UpSampling2D
from keras.models import Sequential, Model
from keras.layers.advanced_activations import LeakyReLU
from keras.datasets import mnist
from keras.optimizers import Adam
from keras.losses import binary_crossentropy

In [43]:
np.random.seed(2332)
randomDim = 100

In [44]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = (X_train.astype(np.float32) - 127.5)/127.5
X_train = X_train[:,:,:,np.newaxis]

In [45]:
X_train.shape

(60000, 28, 28, 1)

In [46]:
adam = Adam(lr=0.0002, beta_1=0.5)

In [59]:
##Let's create the generator

generator=Sequential()
generator.add(Dense(128*7*7,input_dim=randomDim))
generator.add(LeakyReLU(0.2))
generator.add(Reshape((7,7,128)))
generator.add(UpSampling2D((2,2)))
generator.add(Conv2D(64,(5,5),padding='same'))
generator.add(LeakyReLU(0.2))
generator.add(UpSampling2D((2,2)))
generator.add(Conv2D(1,(5,5),padding='same',activation='tanh'))
generator.compile(loss='binary_crossentropy',optimizer=adam)

In [60]:
##discriminator
discriminator=Sequential()
discriminator.add(Conv2D(64,(5,5),padding='same',strides=(2,2)))
discriminator.add(LeakyReLU(0.2))
discriminator.add(Dropout(0.25))
discriminator.add((Conv2D(128,(5,5),padding='same',strides=(2,2))))
discriminator.add(LeakyReLU(0.2))
discriminator.add(Dropout(0.25))
discriminator.add(Flatten())
discriminator.add(Dense(100,activation='relu'))
discriminator.add(Dense(1,activation='sigmoid'))
discriminator.compile(loss='binary,optimizer=adam')


In [61]:
#combine the networks

discriminator.trainable=False
ganInp=Input(shape=(randomDim,))
x=generator(ganInp)     #output from the generator will go in the discriminator
ganout=discriminator(x)

gan=Model(inputs=ganInp,outputs=ganout)
gan.compile(loss='binary_crossentropy',optimizer=adam)

In [62]:
glosses=[]       #generator losses
dlosses=[]        #discriminator losses

In [63]:
def plotloss(epoch):
  plt.figure(figsize=(8,10))
  plt.plot(dlosses,label='Discriminator loss')
  plt.plot(glosses,label='Generator loss')
  plt.xlabel('Epoch')
  plt.ylabel('loss')
  plt.legend()
  plt.savefig('gan%d.png' %epoch)

In [64]:
def plotGeneratedImages(epoch,example=100, dim=(10,10), figsize=(10,10)):
    noise = np.random.normal(0, 1, size=[example, randomDim])
    genImages = generator.predict(noise)
    
    plt.figure(figsize=figsize)
    for i in range(genImages.shape[0]):
        plt.subplot(dim[0], dim[1], i+1)
        plt.imshow(genImages[i,0], cmap='gray')
        plt.axis("off")
    plt.tight_layout()
    plt.savefig('plot%d.png' %epoch)

In [65]:
def train(epochs=1,batchsize=128):
  batchcount=int(X_train.shape[0]/batchsize)
  print("Epochs: %d" %epochs)
  print("Batchsize: %d" %batchsize)
  print("Batches per epoch: %d" %batchcount)

  for e in range(1,epochs+1):

    for _ in tqdm(range(batchcount)):
      noise= np.random.normal(0, 1, size=[batchsize, randomDim])
      imgbatch=X_train[np.random.randint(0,X_train.shape[0],size=batchsize)]
      genimage=generator.predict(noise)
      X=np.concatenate([imgbatch,genimage])
      ydis=np.zeros(2*batchsize)
      ydis[:batchsize]=0.9

      discriminator.trainable=True
      dloss=discriminator.train_on_batch(X,ydis)

      #train generator
      noise=np.random.normal(0,1,size=[batchsize,randomDim])
      ygen=np.ones[batchsize]
      discriminator.trainable=False
      gloss=gan.train_on_batch(noise,ygen)

    dlosses.append(dloss)
    glosses.append(gloss)

    if e==1 or e%10==0:
      plotGeneratedImages(e)

    plotloss(e)






In [66]:
train(20,128)

  0%|          | 0/468 [00:00<?, ?it/s]

Epochs: 20
Batchsize: 128
Batches per epoch: 468


  0%|          | 0/468 [00:00<?, ?it/s]


ValueError: ignored